In [1]:
import pandas as pd
import numpy as np

In [2]:
final_analys_record_personal_access = pd.read_csv('../../../../Data/For_Analysis_and_Modeling/2nd_process/final_analys_teaching_record_personal_access.csv',
       sep='|',
       na_values=['',' ']
    )

In [3]:
final_analys_record_personal_access.head()

,des_plan,anio_apertura_expediente,abandona,convocatoria_acceso,des_acceso,sexo,anio_nacimiento,nota_admision_def
0,GRADO EN EDUCACIÓN SOCIAL,2010-11,0,JUN,Selectividad,D,1992,8.154
1,GRADO EN EDUCACIÓN SOCIAL,2011-12,1,JUN,Selectividad,H,1992,5.682
2,GRADO EN EDUCACIÓN SOCIAL,2012-13,0,JUN,Selectividad,D,1994,11.844
3,GRADO EN EDUCACIÓN SOCIAL,2012-13,0,JUN,Selectividad,D,1994,9.126
4,GRADO EN EDUCACIÓN SOCIAL,2012-13,1,JUN,Selectividad,D,1993,8.118


In [4]:
analys_record_personal_access_one_hot = pd.get_dummies(data=final_analys_record_personal_access, columns=final_analys_record_personal_access.drop(['nota_admision_def','abandona'], axis=1).columns)

In [5]:
analys_record_personal_access_one_hot.shape

(1985, 53)

In [6]:
analys_record_personal_access_one_hot.head()

,abandona,nota_admision_def,des_plan_GRADO EN EDUCACIÓN INFANTIL (CÁCERES),des_plan_GRADO EN EDUCACIÓN PRIMARIA (CÁCERES),des_plan_GRADO EN EDUCACIÓN PRIMARIA. Modalidad Bilingüe (Español-Inglés),des_plan_GRADO EN EDUCACIÓN SOCIAL,anio_apertura_expediente_2009-10,anio_apertura_expediente_2010-11,anio_apertura_expediente_2011-12,anio_apertura_expediente_2012-13,...,anio_nacimiento_1989,anio_nacimiento_1990,anio_nacimiento_1991,anio_nacimiento_1992,anio_nacimiento_1993,anio_nacimiento_1994,anio_nacimiento_1995,anio_nacimiento_1996,anio_nacimiento_1997,anio_nacimiento_1998
0,0,8.154,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1,5.682,0,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
2,0,11.844,0,0,0,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,0,9.126,0,0,0,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,1,8.118,0,0,0,1,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


In [7]:
analys_record_personal_access_one_hot['abandona'].value_counts()

0    1593
1     392
Name: abandona, dtype: int64

In [8]:
from sklearn.utils import resample


In [9]:
# Separate majority and minority classes
df_majority = analys_record_personal_access_one_hot[analys_record_personal_access_one_hot.abandona==0]
df_minority = analys_record_personal_access_one_hot[analys_record_personal_access_one_hot.abandona==1]
 
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    
                                 n_samples=1000,     
                                 random_state=123) 

df_minority_upsampled = resample(df_minority, 
                                 replace=True,
                                 n_samples=1000,    
                                 random_state=123) 
df_modified = pd.concat([df_majority_downsampled, df_minority_upsampled])

In [10]:
df_modified.shape

(2000, 53)

In [11]:
df_modified.abandona.value_counts()

0    1000
1    1000
Name: abandona, dtype: int64

In [12]:
analys_record_personal_access_one_hot_modified = df_modified

In [13]:
X = analys_record_personal_access_one_hot_modified.drop(['abandona'], axis=1)
y = analys_record_personal_access_one_hot_modified['abandona']

In [14]:
from sklearn.model_selection import train_test_split, cross_val_score

In [15]:
import sklearn.metrics as sklm

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=24)

In [17]:
from sklearn.linear_model import LogisticRegression

In [18]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

/home/fran/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [19]:
def print_cv_results (cv_estimate):
    print('Mean performance metric = %4.3f' % np.mean(cv_estimate))
    print('SDT of the metric       = %4.3f' % np.std(cv_estimate))
    print('Outcomes by cv fold')
    for i, x in enumerate(cv_estimate):
        print('Fold %2d    %4.3f' % (i+1, x))

In [20]:
y_pred = clf.predict(X_test)
sklm.accuracy_score(y_true=y_test, y_pred=y_pred)

0.708

In [21]:
sklm.confusion_matrix(y_test, y_pred)

array([[171,  68],
       [ 78, 183]])

In [22]:
tn, fp, fn, tp = sklm.confusion_matrix(y_test, y_pred).ravel()

In [23]:
sklm.recall_score(y_test, y_pred)

0.7011494252873564

In [24]:
tn/(tn+fp)

0.7154811715481172

Esta es la medida que tenemos que tratar de que sea la más alta, porque es preferible predecir un abandono como positivo para prevenirlo, y que luego realmente esa persona no vaya a abandonar, a que se nos escape un abandono.

El hecho de hacer que esta medida aumente, hará que el modelo predizca en más ocasiones un positivo en casos que no lo son, pero es preferible eso y que al modelo se le escapen los número de casos positivos menores posibles, a tratar de balancear los dos lados y que se nos escapen casos positivos.

Cuantos menos casos positivos se nos escapen, mejor, es preferible predecir un falso positivo que no un falso negativo

### Probamos con cv


In [25]:
print_cv_results (cross_val_score(clf, X_train, y_train, scoring='recall'))

/home/fran/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/fran/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n

Mean performance metric = 0.697
SDT of the metric       = 0.052
Outcomes by cv fold
Fold  1    0.696
Fold  2    0.791
Fold  3    0.662
Fold  4    0.696
Fold  5    0.639


/home/fran/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Probamos con el cjto de datos completo

In [26]:
X_test = analys_record_personal_access_one_hot.drop(['abandona'], axis=1)
y_test = analys_record_personal_access_one_hot['abandona']

In [27]:
y_pred = clf.predict(X_test)
sklm.accuracy_score(y_true=y_test, y_pred=y_pred)

0.705793450881612

In [28]:
sklm.confusion_matrix(y_test, y_pred)

array([[1124,  469],
       [ 115,  277]])

In [29]:
sklm.recall_score(y_test, y_pred)

0.7066326530612245

In [30]:
fp/(tn+fp)

0.28451882845188287

### Pasamos a ver la importancia de cada variable en el modelo

In [31]:
import time
from sklearn.inspection import permutation_importance


In [32]:
feature_names = X.columns

In [33]:
start_time = time.time()
result = permutation_importance(
    clf, X_test, y_test, n_repeats=3, random_state=42, n_jobs=-1)
elapsed_time = time.time() - start_time
print(f"Elapsed time to compute the importances: "
      f"{elapsed_time:.3f} seconds")

logit_importances = pd.Series(result.importances_mean, index=feature_names)

Elapsed time to compute the importances: 1.155 seconds


In [34]:
logit_importances = pd.Series(result.importances_mean, index=feature_names)

In [35]:
logit_importances = logit_importances[logit_importances > 0.001]

In [36]:
logit_importances.sort_values()

convocatoria_acceso_EXT                                                      0.001008
anio_nacimiento_1977                                                         0.001008
anio_apertura_expediente_2010-11                                             0.001175
anio_nacimiento_1993                                                         0.001175
anio_apertura_expediente_2016-17                                             0.001343
anio_nacimiento_1988                                                         0.001847
convocatoria_acceso_SEP                                                      0.002183
convocatoria_acceso_JUN                                                      0.003023
des_plan_GRADO EN EDUCACIÓN PRIMARIA. Modalidad Bilingüe (Español-Inglés)    0.003191
anio_nacimiento_1995                                                         0.004198
sexo_D                                                                       0.008060
anio_apertura_expediente_2011-12                      

In [37]:
logit_importances=pd.DataFrame({'Feature':logit_importances.sort_values(ascending=False).index,
              'Permutation_importance':logit_importances.sort_values(ascending=False)}).reset_index(drop=True)

In [38]:
logit_importances

,Feature,Permutation_importance
0,des_plan_GRADO EN EDUCACIÓN PRIMARIA (CÁCERES),0.026196
1,nota_admision_def,0.022166
2,anio_nacimiento_1997,0.019815
3,anio_nacimiento_1998,0.014945
4,anio_apertura_expediente_2012-13,0.014442
5,anio_nacimiento_1996,0.011755
6,anio_nacimiento_1994,0.008396
7,anio_apertura_expediente_2011-12,0.008228
8,sexo_D,0.008060
9,anio_nacimiento_1995,0.004198
